In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import RandomizedSearchCV

from catboost import CatBoostRegressor
from xgboost import XGBRegressor

# Ignore warnings
warnings.filterwarnings("ignore")


In [9]:
data = pd.read_csv('data/exams.csv')

In [10]:
data.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group D,some college,standard,completed,59,70,78
1,male,group D,associate's degree,standard,none,96,93,87
2,female,group D,some college,free/reduced,none,57,76,77
3,male,group B,some college,free/reduced,none,70,70,63
4,female,group D,associate's degree,standard,none,83,85,86


In [12]:
x=data.drop(columns=['math score'],axis =1)

In [14]:
x.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,reading score,writing score
0,female,group D,some college,standard,completed,70,78
1,male,group D,associate's degree,standard,none,93,87
2,female,group D,some college,free/reduced,none,76,77
3,male,group B,some college,free/reduced,none,70,63
4,female,group D,associate's degree,standard,none,85,86


In [15]:
y=data['math score']

In [20]:
num_features = x.select_dtypes(exclude="object").columns
cat_features = x.select_dtypes(include="object").columns
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer
numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()
preprocessor =ColumnTransformer(
[
  ("OneHotEncoder",oh_transformer,cat_features),
  ("StandardScaler",numeric_transformer,num_features),
]

)


In [21]:
x=preprocessor.fit_transform(x)

In [24]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [25]:
def evaluate(true,predicted):
  mae = mean_absolute_error(true,predicted)
  mse = mean_squared_error(true,predicted)
  rmse = np.sqrt(mean_squared_error(true,predicted))
  r2_square =r2_score(true,predicted)
  return mae,rmse,r2_square

In [ ]:
models ={
  "linear_regression":LinearRegression() ,
  "Ridge":Ridge(),
  "Lasso":Lasso(),
  "K-Neighbors Regressir" : KNeighborsRegressor(),
  "Decision Tree":DecisionTreeRegressor(),
  "Random Forest Regressor" : RandomForestRegressor(),
  "xgboost":XGBRegressor(),
  "catboosting regressor":CatBoostRegressor(),
  "adaboost" :AdaBoostRegressor()


}
model_list = []
r2_list = []
for i in range(len(list(models))):
  model = list(models.values())[i]
  model.fit(x_train,y_train)
  x_train_pred = model.predict(x_train)
  y_test_pred = model.predict(x_test)
  model_train_mae,model_rmse,model_train_r2 = evaluate_model(y_train,y_train_pred)
  model_test_mae,model_test_rmse,model_test_r2 = evaluate_model(y_test,y_test_pred)



In [27]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

# ✅ Define evaluation function
def evaluate_model(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    return mae, rmse, r2


# ✅ Define models
models = {
    "Linear Regression": LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBoost": XGBRegressor(verbosity=0),
    "CatBoost Regressor": CatBoostRegressor(verbose=0),
    "AdaBoost": AdaBoostRegressor(),
}

# ✅ Store results
results = []

for name, model in models.items():
    model.fit(x_train, y_train)

    # Train & test predictions
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    # Evaluate
    train_mae, train_rmse, train_r2 = evaluate_model(y_train, y_train_pred)
    test_mae, test_rmse, test_r2 = evaluate_model(y_test, y_test_pred)

    results.append({
        "Model": name,
        "Train R2": train_r2,
        "Test R2": test_r2,
        "Train RMSE": train_rmse,
        "Test RMSE": test_rmse,
        "Train MAE": train_mae,
        "Test MAE": test_mae
    })

# ✅ Convert to DataFrame for comparison
results_df = pd.DataFrame(results)
print(results_df)


                     Model  Train R2   Test R2  Train RMSE  Test RMSE  \
0        Linear Regression  0.872328  0.887488    5.471957   4.983830   
1                    Ridge  0.872320  0.887328    5.472140   4.987369   
2                    Lasso  0.807400  0.789992    6.720846   6.808984   
3    K-Neighbors Regressor  0.860500  0.810224    5.719830   6.472681   
4            Decision Tree  1.000000  0.746154    0.000000   7.485987   
5  Random Forest Regressor  0.976718  0.853453    2.336733   5.687907   
6                  XGBoost  0.996679  0.826102    0.882480   6.195991   
7       CatBoost Regressor  0.957570  0.874470    3.154504   5.264256   
8                 AdaBoost  0.839721  0.832163    6.131042   6.087066   

   Train MAE  Test MAE  
0   4.389094  4.073537  
1   4.388289  4.073221  
2   5.457572  5.375728  
3   4.530250  5.150000  
4   0.000000  5.920000  
5   1.831950  4.591250  
6   0.606247  5.067168  
7   2.446369  4.144307  
8   5.052432  4.962117  
